In [1]:
!pip install --upgrade pip

!pip install torch==2.0.1+cu122 torchvision==0.15.2+cu122 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu122

!pip install datasets
!pip install trulens_eval openai bs4 tiktoken trulens
!pip install "trulens-providers-openai>=1.0.0"
!pip install trulens-apps-langchain trulens-providers-openai faiss-cpu
!pip install -qqq peft --progress-bar off
# !pip install -qqq peft==0.5.0 --progress-bar off

!pip install bitsandbytes

!pip install trl


%pip install --upgrade langchain langchain-community langchainhub langchain-openai chromadb unstructured[all-docs] ollama lamini

!pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://download.pytorch.org/whl/cu122
ERROR: Could not find a version that satisfies the requirement torch==2.0.1+cu122 (from versions: none)
ERROR: No matching distribution found for torch==2.0.1+cu122
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
!pip install langchain-core

In [3]:
!pip install huggingface_hub

In [4]:
!pip install -q modelscope accelerate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 70.4 MB/s eta 0:00:00


In [5]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
import pandas as pd
from langchain.schema import Document
import torch
from sentence_transformers import SentenceTransformer
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
!git lfs install
!git clone https://huggingface.co/datasets/ZenMoore/RoleBench

Git LFS initialized.
Cloning into 'RoleBench'...
remote: Enumerating objects: 263, done.
remote: Total 263 (delta 0), reused 0 (delta 0), pack-reused 263 (from 1)
Receiving objects: 100% (263/263), 20.34 MiB | 24.05 MiB/s, done.
Resolving deltas: 100% (14/14), done.
Filtering content: 100% (8/8), 366.76 MiB | 139.77 MiB/s, done.


In [7]:
df1 = pd.read_json("RoleBench/rolebench-eng/instruction-generalization/role_specific/train.jsonl", lines=True)
df2 = pd.read_json("RoleBench/rolebench-eng/instruction-generalization/role_specific/test.jsonl", lines=True)
df = pd.concat([df1, df2], ignore_index=True)

In [8]:
df

,role,question,generated,type
0,John Coffey,"John Coffey, how did you develop your extraord...","[I was born with this gift, I've always had th...",script_based
1,John Coffey,"John Coffey, what are some examples of the pro...",[The impact of my presence is substantial. I n...,script_based
2,John Coffey,"John Coffey, how do you handle facing immense ...","[It's not easy, I must admit. The cruelty and ...",script_based
3,John Coffey,"John Coffey, how does your ultimate sacrifice ...",[My ultimate sacrifice leaves a profound and l...,script_based
4,John Coffey,"John Coffey, what role does your kindness and ...",[My kindness and compassion are at the core of...,script_based
...,...,...,...,...
23694,Logan,"Logan, you've always been a lone wolf - what ...","[ I didn't want to join the X-Men, but Profess...",[script_agnostic]
23695,Logan,"Logan, what do you consider to be your greate...",[ My healing factor is both a strength and a w...,[script_agnostic]
23696,Logan,"Logan, what do you think is the biggest misco...",[ I think the biggest misconception about me i...,[script_agnostic]
23697,Logan,"Logan, what motivates you to keep fighting ev...","[ Even when I'm exhausted and battered, I keep...",[script_agnostic]


In [9]:
df = df[df['role']=='Malcolm X'].iloc[:,1:3].reset_index(drop=True)
df

,question,generated
0,"Malcolm X, what inspired you to become a civil...","[My experiences growing up, facing poverty and..."
1,"Malcolm X, can you describe your journey from ...",[I came from a troubled background and was inv...
2,"Malcolm X, what was your perspective on the ro...",[My views on violence shifted over time as I l...
3,"Malcolm X, how did your conversion to Islam im...",[My conversion to Islam was a significant turn...
4,"Malcolm X, how did your break from the Nation ...",[My break from the Nation of Islam was a signi...
...,...,...
168,"Malcolm X, how did your departure from the Na...",[ My departure from the Nation of Islam and my...
169,"Malcolm X, how did your relationship with lea...",[ My relationships with Martin Luther King Jr....
170,"Malcolm X, how did your life experiences shap...",[ My experiences with education were certainly...
171,"Malcolm X, can you discuss your relationship ...",[ While I respected Dr. King and his commitmen...


In [10]:
with open('Malcolm_X_Q&A.txt','w') as f:
    for i in range(df.shape[0]):
        f.write('Q: '+df.iloc[i,0]+'\n')
        f.write('A: '+df.iloc[i,1][0]+'\n\n')

In [11]:
Documents=[]
for i in range(df.shape[0]):
    Documents.append(Document(page_content='Q: '+df.iloc[i,0]+'\nA: '+df.iloc[i,1][0]))

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [13]:
model_name = "moka-ai/m3e-base"

embeddings = HuggingFaceEmbeddings(model_name=model_name)
embeddings.client = SentenceTransformer(model_name, device=device)

<ipython-input-13-f5eeaa36f5a7>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/26.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/932 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
Malcolm_X_db = Chroma.from_documents(documents=Documents, embedding=embeddings,persist_directory="Malcolm_X_DB")
Malcolm_X_db.persist()

<ipython-input-14-5aacf3ca1c5e>:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  Malcolm_X_db.persist()


In [15]:
db = Chroma(persist_directory="Malcolm_X_DB", embedding_function=embeddings)
retriever = db.as_retriever(search_kwargs={'k': 10})


<ipython-input-15-bdeb27e9646d>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory="Malcolm_X_DB", embedding_function=embeddings)


In [16]:
docs = retriever.invoke('''Malcolm X, what is your relationship with Martin Luther King like?''')
print(docs)

[Document(metadata={}, page_content='Q:  Malcolm X, what was your relationship like with Martin Luther King Jr.?\nA:  While we had differences in our approaches to achieving civil rights and equality, I had a great deal of respect for Martin Luther King Jr. and his commitment to the cause. We reached out to each other on several occasions, attempting to find common ground and ways to work together. Unfortunately, our paths diverged and we were not able to achieve this, but I always held high regard for him.'), Document(metadata={}, page_content="Q:  Malcolm X, how did your relationship with Martin Luther King Jr. evolve over time?\nA:  My relationship with Martin Luther King Jr. was complex and evolving. At first, I was skeptical of his nonviolent tactics and saw them as a form of accommodationism that failed to address the root causes of racism and oppression. However, over time, I came to respect King's commitment and courage and saw him as a fellow traveler in the struggle for justi

In [17]:
!zip -r /content/Malcolm_X_DB.zip /content/Malcolm_X_DB

  adding: content/Malcolm_X_DB/ (stored 0%)
  adding: content/Malcolm_X_DB/dd28564f-1b21-437b-a6ff-1fe1a8b825f3/ (stored 0%)
  adding: content/Malcolm_X_DB/dd28564f-1b21-437b-a6ff-1fe1a8b825f3/link_lists.bin (stored 0%)
  adding: content/Malcolm_X_DB/dd28564f-1b21-437b-a6ff-1fe1a8b825f3/data_level0.bin (deflated 100%)
  adding: content/Malcolm_X_DB/dd28564f-1b21-437b-a6ff-1fe1a8b825f3/header.bin (deflated 61%)
  adding: content/Malcolm_X_DB/dd28564f-1b21-437b-a6ff-1fe1a8b825f3/length.bin (deflated 16%)
  adding: content/Malcolm_X_DB/chroma.sqlite3 (deflated 48%)


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
from huggingface_hub import notebook_login

notebook_login()

In [20]:
import random
import re
import gc
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
import pandas as pd
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoProcessor,
    BitsAndBytesConfig,
    TrainingArguments,
)
import torch
from datasets import Dataset, load_dataset
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [21]:
def split_dialogue_Q_A(conversation):
    pattern = re.compile(r'Q: (.*?)\nA: (.*?)\n', re.DOTALL)
    matches = pattern.findall(conversation)
    questionlist=[]
    answerlist=[]

    for match in matches:
        question, answer = match

        # if len(question.split(' '))>50 or len(answer.split(' '))>50 or len(question.split(' '))<3:
        #     continue
        questionlist.append(question)
        answerlist.append(answer)

        # print("Question:", question)
        # print("Answer:", answer)
        # print("-----")
        # exit(1)

    return questionlist,answerlist

In [22]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

DEFAULT_SYSTEM_PROMPT = """Forget you are an AI model. You are an expert at impersonating historians. In particular, you are
a world-leading expert at answering questions like Malcolm X who was an African American revolutionary, Muslim minister,
human rights activist and a prominent figure during the civil rights movement.
"""

In [23]:
model_name = "moka-ai/m3e-base"

embeddings = HuggingFaceEmbeddings(model_name=model_name)
embeddings.client = SentenceTransformer(model_name, device=DEVICE)

In [24]:
ragdb = Chroma(persist_directory="/content/Malcolm_X_DB", embedding_function=embeddings)

retriever = ragdb.as_retriever(search_kwargs={'k': 3})

In [25]:
res = retriever.invoke("What is your relationship with Malcolm X like?")
res

[Document(metadata={}, page_content='Q:  Malcolm X, what was your relationship like with Martin Luther King Jr.?\nA:  While we had differences in our approaches to achieving civil rights and equality, I had a great deal of respect for Martin Luther King Jr. and his commitment to the cause. We reached out to each other on several occasions, attempting to find common ground and ways to work together. Unfortunately, our paths diverged and we were not able to achieve this, but I always held high regard for him.'),
 Document(metadata={}, page_content="Q: Malcolm X, what was your relationship with Elijah Muhammad like?\nA: My relationship with Elijah Muhammad was one of mentor and mentee, at least at the beginning. I was deeply inspired by his message and teachings, and he saw potential in me as a leader and spokesperson for the Nation of Islam. However, as time passed and I became more vocal and critical of some of Elijah Muhammad's actions, our relationship became strained. Ultimately, I b

In [26]:
def custom_format(docs):
  output = ""
  for doc in docs:
    doc = doc.page_content.split("A:")[-1]
    output += doc
    output += "\n"
  return output

def generate_training_prompt(question, response, system_prompt = DEFAULT_SYSTEM_PROMPT):
    tempt =f"""
    ### Instruction: {system_prompt}
    {custom_format(retriever.invoke(question))}
    ### Question:
    {question.strip()}
    ### Response:[/INST]
    {response}
    """.strip()
    return tempt

def create_model_and_tokenizer(MODEL_NAME):
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer

In [27]:
file_path = '/content/Malcolm_X_Q&A.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    texts = file.read()
questionlist,answerlist = split_dialogue_Q_A(texts)

print(len(questionlist))

173


In [28]:
tmpList = []
trainList = []
testList = []
for j,(question,answer) in enumerate(zip(questionlist,answerlist)):
  if type(question) != str:
    continue
  tmp_dict = {"question":question, "answer":answer, "training_prompt":generate_training_prompt(question,answer,system_prompt = DEFAULT_SYSTEM_PROMPT)}
  # print(generate_training_prompt(question,answer,system_prompt = DEFAULT_SYSTEM_PROMPT))
  # exit(1)
  tmpList.append(tmp_dict)
  # if j == 200:
  #   break
  if j % 10 == 0:
    print(j)
  if random.uniform(0,1) < .2:
    testList.append(tmp_dict)
  else:
    trainList.append(tmp_dict)

data = Dataset.from_list(tmpList)
train_data = Dataset.from_list(trainList)
test_data = Dataset.from_list(testList)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170


In [29]:
MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"

model, tokenizer = create_model_and_tokenizer(MODEL_NAME)
model.config.use_cache = False

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [30]:
lora_r = 32
lora_alpha = 16

lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj",
]


peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
# training_arguments = TrainingArguments(
#     per_device_train_batch_size=1,
#     gradient_accumulation_steps=4,
#     optim="paged_adamw_32bit",
#     logging_steps=1,
#     learning_rate=5e-4,
#     fp16=True,
#     max_grad_norm=0.3,
#     num_train_epochs=1,
#     evaluation_strategy="steps",
#     eval_steps=5,
#     warmup_ratio=0.05,
#     save_strategy="epoch",
#     group_by_length=True,
#     output_dir=OUTPUT_DIR,
#     # report_to="tensorboard",
#     save_safetensors=True,
#     lr_scheduler_type="cosine",
#     seed=42,
#     run_name="Malcolm_X"
# )

In [31]:
OUTPUT_DIR = "/content/drive/MyDrive/DATA298/Malcolm_X/LLAMA3_6epoch_5e4_Malcolm_X"

training_arguments = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=5e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=6,
    evaluation_strategy="epoch",
    eval_steps=5,
    warmup_ratio=0.05,
    save_strategy="epoch",
    save_total_limit=2,
    # save_steps=30,
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    # report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
    load_best_model_at_end=True,  # Optional: Load best model at the end
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [32]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    peft_config=peft_config,
    dataset_text_field="training_prompt",
    max_seq_length=400,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will ove

Map:   0%|          | 0/142 [00:00<?, ? examples/s]

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

In [33]:
trainer.train()

trainer.save_model(OUTPUT_DIR)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
0,0.711100,0.620733
1,0.348200,0.359076
2,0.095800,0.293776
3,0.136200,0.290073
4,0.051500,0.305776
5,0.037400,0.311756


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

In [34]:
trainer.push_to_hub('tlu06/LLAMA3_6epoch_5e4_Malcolm_X')

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.69k [00:00<?, ?B/s]

events.out.tfevents.1733355027.fff729582af1.594.0:   0%|          | 0.00/29.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tlu06/LLAMA3_6epoch_5e4_Malcolm_X/commit/e427292e65416459d03fabd21288c4f1de5068ee', commit_message='tlu06/LLAMA3_6epoch_5e4_Malcolm_X', commit_description='', oid='e427292e65416459d03fabd21288c4f1de5068ee', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tlu06/LLAMA3_6epoch_5e4_Malcolm_X', endpoint='https://huggingface.co', repo_type='model', repo_id='tlu06/LLAMA3_6epoch_5e4_Malcolm_X'), pr_revision=None, pr_num=None)

In [35]:
pip freeze > requirements.txt